In [1]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [2]:
load_dotenv(override=True)

True

In [3]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

In [4]:
search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [5]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

In 2025, several AI agent frameworks have emerged, each offering unique capabilities:

- **Agentforce 360**: Salesforce's comprehensive platform for building enterprise AI agents, featuring Agentforce Builder for natural language-based agent creation, Agent Script for defining agent behavior, and Agentforce Voice for realistic phone interactions. ([itpro.com](https://www.itpro.com/technology/artificial-intelligence/salesforce-just-launched-a-new-catch-all-platform-to-build-enterprise-ai-agents?utm_source=openai))

- **Agent Lightning**: A flexible framework enabling reinforcement learning-based training of large language models for any AI agent, supporting integration with existing agents with minimal code modifications. ([arxiv.org](https://arxiv.org/abs/2508.03680?utm_source=openai))

- **Cognitive Kernel-Pro**: An open-source, multi-module agent framework designed to democratize the development and evaluation of advanced AI agents, achieving state-of-the-art results among open-source agents. ([arxiv.org](https://arxiv.org/abs/2508.00414?utm_source=openai))

- **AutoAgent**: A fully-automated, zero-code framework that allows users to create and deploy large language model agents through natural language alone, serving as a versatile multi-agent system for general AI assistants. ([arxiv.org](https://arxiv.org/abs/2502.05957?utm_source=openai))

- **Amazon Bedrock AgentCore**: AWS's platform designed to simplify the development and deployment of advanced AI agents, including features like AgentCore Runtime for scalable serverless deployment and AgentCore Memory for context management. ([techradar.com](https://www.techradar.com/pro/aws-looks-to-super-charge-ai-agents-with-amazon-bedrock-agentcore?utm_source=openai))

- **Model Context Protocol (MCP)**: An open standard introduced by Anthropic to standardize the integration and sharing of data between AI systems and external tools, adopted by major AI providers including OpenAI and Google DeepMind. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Model_Context_Protocol?utm_source=openai))

- **Kruti**: A multilingual AI agent developed by Ola Krutrim, designed to perform real-world tasks for users, such as booking taxis and ordering food, by integrating directly with various online services. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Kruti?utm_source=openai))

- **Manus**: An autonomous AI agent developed by Butterfly Effect Technology, capable of independent reasoning, dynamic planning, and autonomous decision-making. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Manus_%28AI_agent%29?utm_source=openai))

These frameworks reflect the rapid advancements in AI agent development, offering diverse tools and platforms for creating intelligent, autonomous systems. 

In [6]:
HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"
# With massive thanks to student Wes C. for discovering and fixing a nasty bug with this!

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")

    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [7]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To find the latest trends and frameworks for AI agents that are emerging in 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To get information on specific tools and technologies that are being used for AI agents this year.', query='top AI agent technologies 2025'), WebSearchItem(reason='To learn about advancements in AI agent frameworks and their applications in various industries in 2025.', query='AI agent frameworks applications 2025')]


In [ ]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ed@edwarddonner.com") # Change this to your verified email
    to_email = To("ed.donner@gmail.com") # Change this to your email
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}